This Jupyter notebook describes the use of the **reaction** script, which is mainly used to quickly calculate the energy change of each reaction step  
- Catalytic_Post_Process: Read adsorption energy calculation results from VASP and calculate reaction energy change


In [1]:
from Jworkflow.reaction import Catalytic_Post_Process

# Catalytic_Post_Process
## Screening the most stable structure and calculating the reaction energy
The script is used to select the most stable configuration from a series of adsorption calculations, and analyze the energy change in the reaction process

First, the adsorption energy calculation results(adss: adsorption structures), slab energy calculation results and possible free energy correction results stored in excel files are read. Put them in the same folder and read them. It is best to generate these files through the **data_process** script.

In [2]:
CPP = Catalytic_Post_Process()
CPP.read_excel('adss.xlsx', 'slab.xlsx', 'Gcor.xlsx', file_path=r'example\data_process&reaction')

--------------------------------Read Data--------------------------------
                          Adss Data : Read Complete - (42, 16)
                          Slab Data : Read Complete - (3, 9)
                          Gcor Data : Read Complete - (15, 8)
                          Task type : calculate energy and G correction
-----------------------------Read Compelete------------------------------


The result read by **data_process** contains some information about the movement of the surface or adsorbate atoms, the convergence of the calculation, etc., according to this information, the calculation result can be initially filtered. If the calculation result is correct, skip this step.

In this function, two criteria are defined. The first one, **criteria_broken**, is used to determine the structures that are broken and need to be discarded. The second one, **criteria_inspect**, is for structures that have some issues but it's not certain whether they should be finally accepted. In the example provided, results where the adsorbate molecule moves too far upwards (not adsorbed) or the bond length of the adsorbate is too large (molecular decomposition) are discarded. Results with significant surface changes or calculations that have not converged are considered for further inspection.

In [3]:
CPP.filter_by_criteria(criteria_broken="(df['mupgrade_skel'] > 1.5) | (df['mdista_adsb'] > 0.6)", 
                       criteria_inspect="(df['mshift_slab'] > 0.8) | (df['converg'] == False)")

---------------------------Filter by Criteria----------------------------
                        Data total  : 42
                        Data broken : 7
                       Data inspect : 5
                        Data remain : 30
------------------------------Filter Finish------------------------------


After the problematic structure is excluded, the most stable adsorption configuration on each slab and each adsorbate is found in the remaining structures. Here, the list of adsorbates contained in reactions and the list of corresponding reactions are passed in. Then the step energy of the corresponding adsorbates are calculated according to reaction formulas, which can be queried, modified or added in the **dataset** script.

In [4]:
CPP.find_stablest_adss(['N2', 'NNH', 'NH', 'NH2', 'NH3'], ['NRR'])

--------------------------Calculate Ads Energy---------------------------
                          Adsb_list : ['N2', 'NNH', 'NH', 'NH2', 'NH3']
                     Reaction_types : ['NRR']
                      System number : 3
                        Pair number : 15
                     Nan pair count : 0
                   Calculated count : 15
--------------------------Calculation Compelete--------------------------


Once the most stable structures and their energies have been obtained, the following commands can be used to find out if there are more stable energies in the structures previously considered for further inspection, and if any, further examination of these structures is recommended.

In [5]:
CPP.inspecting_energy()

----------------------------Inspection Energy----------------------------
                       Inspect adss :    E_inspect       E_lower
                   Ga3Sc_NNHh_90_1t : -87.28278283 -0.4113317000000052
-----------------------------Inspection Done-----------------------------


After finding the adsorption configuration with the most stable energy of each adsorbate, the potential barrier of the reaction can be calculated. Here, a list of three dimensions is needed to define the reaction path, and the program will query the energy change of the adjacent two in each list as the potential barrier.

Multiple third-level lists in the second-level list represent different reaction paths，like[ [ [ 'NNH', 'NNH2' ], [ 'NNH', 'NHNH' ] ] ].

In [6]:
CPP.find_barrier([[['N2', 'NNH']], [['NH', 'NH2', 'NH3']]])

----------------------------Find_Barrier_PDS-----------------------------
                        Max barrier : Ga3Sc - N2->NNH - 1.4027814250000006
                        Min barrier : Sn3Y - N2->NNH - 1.3088210049999986
                          PDS count : N2->NNH: 3
                 Zreo barrier count : 0
-----------------------------------END-----------------------------------


After calculating adsortion energies, the linear relationship of adsorption energy between each adsorbent can be quickly fitted and viewed.

In [7]:
CPP.fit_linear_relation()

---------------------------Fit_Linear_Relation---------------------------
                    Relation number : 20
                       Max R² score : N2_NH2 - 0.9920781458448504
                       Min R² score : NH_N2 - 0.009943813198988405
             Percentage of R² > 0.9 :  4 / 20
-----------------------------------END-----------------------------------


In [8]:
CPP.df_lr.at['N2', 'NNH']

{'x': 'N2',
 'y': 'NNH',
 'k': 0.22919545234410915,
 'b': 1.300741002907515,
 'R2': 0.17016808219443602}

If desired, an energy of the end product can be added, where the two inputs represent the reaction and the name of the end product. This requires the product to be defined in the **dataset** script.

In [9]:
CPP.add_prod('NRR', 'NH3(g)')

--------------------------------Add Prod---------------------------------
                             NH3(g) : -0.6390999400000013
-----------------------------------END-----------------------------------


Check the **df_path_step** to see the energy, barrier, PDS and other information of each adsorbate corresponding to each slab.

In [10]:
CPP.df_path_step

,N2,NNH,NH,NH2,NH3,barrier,PDS,NH3(g)
Ga3Sc,-0.145807,1.256974,0.573475,-0.62796,-1.110687,1.402781,N2->NNH,-0.6391
ScYTiGaGeSn,-0.073361,1.315445,-0.670773,-0.815012,-1.203226,1.388806,N2->NNH,-0.6391
Sn3Y,-0.037947,1.270874,0.71966,-0.876187,-1.202499,1.308821,N2->NNH,-0.6391


**export_adss_names** can be used to output the names of structures that require further inspection, or that have the lowest energy, for further post-processing.

In [11]:
CPP.export_adss_names('inspect', 'str_list_inspect')

-------------------------------Export adss-------------------------------
                          Save file : example\data_process&reaction\str_list_inspect
-----------------------------------END-----------------------------------


In [12]:
CPP.export_adss_names('Emin', 'str_list_emin')

-------------------------------Export adss-------------------------------
                          Save file : example\data_process&reaction\str_list_emin
-----------------------------------END-----------------------------------


## Calculated reaction energy
If there is no need to screen the configuration, the script can also be used to quickly calculate the adsorption energy according to the formula. After reading the calculation results, calculate all adsorption energies using **calculate_energy_all** and finally view the results in **df_adss**. reactions can be viewed, modified, or added in **dataset** scripts.

In [13]:
CPP = Catalytic_Post_Process()
CPP.read_excel('adss.xlsx', 'slab.xlsx', 'Gcor.xlsx', file_path=r'example\data_process&reaction')

--------------------------------Read Data--------------------------------
                          Adss Data : Read Complete - (42, 16)
                          Slab Data : Read Complete - (3, 9)
                          Gcor Data : Read Complete - (15, 8)
                          Task type : calculate energy and G correction
-----------------------------Read Compelete------------------------------


In [14]:
CPP.calculate_energy_all(['NRR', 'HER'])

--------------------------Calculate Energy All---------------------------
                        Calculation : Complete
                   Calculated count : 15
                      Ignored count : 27
-----------------------------------END-----------------------------------


In [15]:
CPP.df_adss.loc[:, ['ads_sys', 'energy']]

,ads_sys,energy
0,Ga3Sc_N2v_0_0t,-0.145807
1,Ga3Sc_NH_0_1h,0.573475
2,Ga3Sc_NH2_45_0t,-0.627960
3,Ga3Sc_NH3_0_0t,-1.110687
4,Ga3Sc_NNHv_0_0t,1.256974
5,ScYTiGaGeSn_N2v_0_0t,-0.073361
6,ScYTiGaGeSn_NH_0_1h,-0.670773
7,ScYTiGaGeSn_NH2_45_0t,-0.815012
8,ScYTiGaGeSn_NH3_0_0t,-1.203226
9,ScYTiGaGeSn_NNHh_45_0t,1.315445


In [16]:
CPP.export_df(CPP.df_adss.loc[:, ['ads_sys', 'adsb', 'site', 'energy']], 'Gcal.xlsx')

----------------------------Export DataFrame-----------------------------
                         Save excel : example\data_process&reaction\Gcal.xlsx
-----------------------------------END-----------------------------------
